1. detection : 사람, 총, 칼 (epoch180.pt 사용)
2. tracking id 생성
3. segmentation : 사람 한정
4. Arcface : tracking id 당 한 번씩만 (Me, NotME)
5. notme, 사람과 무기의 bounding box가 겹치는 경우 위험인 분류 -> 위험인의 id는 dangerous_ids로 관리됨 ("dangerous person" 표시)
   (꼬부기는 dangerous person을 추적)
7. pose estimation (왼팔들기, 오른팔들기, 양팔들기)  

v6: dangerous person 관리하도록 수정


In [1]:
#1. 얼굴 학습시키는 부분
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis

def initialize_arcface():
    app = FaceAnalysis(name="buffalo_l")  # ArcFace 모델 (buffalo_l은 기본 권장)
    app.prepare(ctx_id=-1, det_size=(640, 640))  # GPU: ctx_id=0, CPU: -1
    return app

def get_face_embedding(app, image_bgr):
    # ArcFace의 app.get()은 BGR 형식으로 이미지를 받기도 합니다.
    # 만약 RGB가 필요하면 cvtColor로 변환하세요.
    faces = app.get(image_bgr)
    if len(faces) > 0:
        return faces[0].embedding  # 첫 번째 얼굴의 임베딩
    else:
        return None

def generate_average_embedding(app, folder_path):
    embeddings = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"이미지 로드 실패: {img_path}")
                continue
            
            embedding = get_face_embedding(app, image)
            if embedding is not None:
                embeddings.append(embedding)
            else:
                print(f"얼굴 검출 실패: {img_path}")
    
    if len(embeddings) == 0:
        raise ValueError("임베딩을 하나도 생성하지 못했습니다.")
    
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

if __name__ == "__main__":
    app = initialize_arcface()
    # 내 얼굴 사진 폴더
    my_face_folder = "C:/myface"  
    my_face_embedding = generate_average_embedding(app, my_face_folder)
    np.save("my_face_embedding.npy", my_face_embedding)  # 필요 시 저장
    print("내 얼굴 평균 임베딩 생성 완료.")

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


내 얼굴 평균 임베딩 생성 완료.


In [2]:
import cv2
import numpy as np
import time
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
import mediapipe as mp

############################
# 1) 모델 및 함수 초기화
############################
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# YOLO Detection 모델 로드
detection_model = YOLO("C:/epoch180.pt").to(device)

# YOLO Segmentation 모델 로드
segmentation_model = YOLO("yolov8n-seg.pt").to(device)

# DeepSORT 초기화
tracker = DeepSort(max_age=30, n_init=3, embedder='mobilenet', half=True, embedder_gpu=True)

# ArcFace 초기화
arc_app = FaceAnalysis(name="buffalo_l")
arc_app.prepare(ctx_id=-1, det_size=(640, 640))
my_face_embedding = np.load("my_face_embedding.npy")

# MediaPipe Pose Estimation 초기화
mp_pose = mp.solutions.pose
pose_model = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# ArcFace 분석 결과 저장
arcface_results = {}  # {track_id: {"is_me": bool, "similarity": float}}

# Dangerous person 관리
dangerous_ids = set()

def get_face_embedding(face_image, arc_app):
    faces = arc_app.get(face_image)
    if len(faces) > 0:
        return faces[0].embedding
    return None

def is_my_face(face_embedding, my_face_embedding, threshold=0.4):
    if face_embedding is None:
        return False, 0.0
    similarity = cosine_similarity([face_embedding], [my_face_embedding])[0][0]
    return similarity > threshold, similarity

def boxes_overlap(boxA, boxB):
    x1A, y1A, x2A, y2A = boxA
    x1B, y1B, x2B, y2B = boxB
    return not (x2A < x1B or x2B < x1A or y2A < y1B or y2B < y1A)

############################
# 2) 실시간 웹캡 캡처
############################
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

window_name = "YOLOv8 + DeepSORT + Dangerous Person"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 960, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    # YOLOv8 Detection 실행
    detection_results = detection_model(frame)
    detections = []
    weapon_boxes = []  # 무기 위치 저장

    for box in detection_results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls)
        conf = float(box.conf)
        label = f"{detection_model.names[class_id]}: {conf:.2f}"

        if class_id == 0:  # 사람
            color = (0, 255, 0)  # 초록색
            detections.append(((x1, y1, x2 - x1, y2 - y1), conf, 0))
        elif class_id in [1, 2]:  # 무기 (총, 칼)
            color = (0, 0, 255)  # 빨간색
            weapon_boxes.append((x1, y1, x2, y2))  # 무기 위치 저장
        else:
            continue

        # 바운딩 박스 그리기
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # DeepSORT 추적
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())

        # Tracking ID 표시
        cv2.putText(frame, f"ID:{track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        # ArcFace (한 번만 실행)
        if track_id not in arcface_results:
            person_crop = frame[y1:y2, x1:x2]
            face_embedding = get_face_embedding(person_crop, arc_app)
            is_me, similarity = is_my_face(face_embedding, my_face_embedding)
            arcface_results[track_id] = {"is_me": is_me, "similarity": similarity}

        arcface_data = arcface_results[track_id]
        if arcface_data["is_me"]:
            arc_text = f"Me (sim={arcface_data['similarity']:.2f})"
            cv2.putText(frame, arc_text, (x1, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            arc_text = f"Not Me (sim={arcface_data['similarity']:.2f})"
            cv2.putText(frame, arc_text, (x1, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Dangerous person 체크
        person_box = (x1, y1, x2, y2)
        for weapon_box in weapon_boxes:
            if boxes_overlap(person_box, weapon_box):
                dangerous_ids.add(track_id)
                break

        if track_id in dangerous_ids:
            cv2.putText(frame, "Dangerous Person", (x1, y1 + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Pose Estimation (팔 들기 감지)
        person_crop_rgb = cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2RGB)
        pose_result = pose_model.process(person_crop_rgb)

        if pose_result.pose_landmarks:
            landmarks = pose_result.pose_landmarks.landmark
            left_wrist_y = landmarks[15].y
            right_wrist_y = landmarks[16].y
            left_shoulder_y = landmarks[11].y
            right_shoulder_y = landmarks[12].y

            if left_wrist_y < left_shoulder_y - 0.05:
                cv2.putText(frame, "Left arm up", (x1, y1 + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            if right_wrist_y < right_shoulder_y - 0.05:
                cv2.putText(frame, "Right arm up", (x1, y1 + 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)


    
    # FPS 계산
    current_time = time.time()
    fps = 1.0 / (current_time - time.time())
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 화면 표시
    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

ZeroDivisionError: float division by zero

In [ ]:
import torch

print("CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())